Classification Problem:
Problem:input sentence / output English/Spanish
Model:Bag Of Vectors - Bag Of Words (BOW) #hint:not cbow
Details:
This model not using any pretrained vectors (word2vec nor glove)
It starts from very scratch dataset with one hot encoding
#ref: https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html#sphx-glr-beginner-nlp-deep-learning-tutorial-py


<img src="BagOfWords_Model.jpg">

In [1]:
import torch 
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
torch.manual_seed(1)

In [2]:
device = torch.device("cuda:0")
print(torch.cuda.get_device_properties(device))
print(device)
#however iam using cpu in that model it's not big dataset

_CudaDeviceProperties(name='GeForce RTX 2060', major=7, minor=5, total_memory=6144MB, multi_processor_count=30)
cuda:0


In [3]:
train_data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

In [4]:
#word to index each word in data
word_to_idx = {}
for sentance,_ in train_data+test_data:
    for word in sentance:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
print(word_to_idx)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [5]:
vocab_size = len(word_to_idx)
num_of_labels = 2

In [6]:
class BoWClassifier(nn.Module):
    def __init__(self,input_size,output_size):
        super(BoWClassifier,self).__init__()
        self.linear = nn.Linear(input_size,output_size)
    def forward(self,bow_vector):
        #output = torch.log_softmax(self.linear(bow_vector),dim=1)
        output = F.log_softmax(self.linear(bow_vector), dim=1)
        return output

In [7]:
class dataset(Dataset):
    def __init__(self,data,word_to_idx,label_to_ix):
        self.x = []
        self.y = []
        self.word_to_idx = word_to_idx
        for i,(sentance,label) in enumerate(data):
            self.x.append(sentance)
            self.y.append(label_to_ix[label])
        
    def __getitem__(self,idx):       
        target = torch.tensor(self.y[idx]).view(-1)
        sentance = self.x[idx]
        one_hot_vec = torch.zeros(len(sentance),len(self.word_to_idx))
        for i,word in enumerate(sentance):
            one_hot_vec[i,self.word_to_idx[word]] = 1
        bag_vector = one_hot_vec.sum(0)
        '''
        #you can replace last 4 line with next 3 lines but however uncomment part is general. 
        one_hot_vec = torch.zeros(len(self.word_to_idx))
        for word in sentance:
            one_hot_vec[self.word_to_idx[word]] += 1
        '''
        #take average of one_hot_vec for sentence
        
        return bag_vector.view(1, -1),target
    
    def __len__(self):
        return len(self.word_to_idx)

In [8]:
model = BoWClassifier(vocab_size,num_of_labels)

In [10]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1011, -0.0866, -0.0380,  0.0921, -0.1846,  0.1176, -0.0403,  0.0998,
          0.0273, -0.0240,  0.0544,  0.0097,  0.0716, -0.0764, -0.0143, -0.0177,
          0.0284, -0.0008,  0.1714,  0.0610, -0.0730, -0.1184, -0.0329, -0.0846,
         -0.0628,  0.0094],
        [ 0.1169,  0.1066, -0.1917,  0.1216,  0.0548,  0.1860,  0.1294, -0.1787,
         -0.1865, -0.0946,  0.1722, -0.0327,  0.0839, -0.0911,  0.1924, -0.0830,
          0.1471,  0.0023, -0.1033,  0.1008, -0.1041,  0.0577, -0.0566, -0.0215,
         -0.1885, -0.0935]], requires_grad=True)
Parameter containing:
tensor([ 0.1064, -0.0477], requires_grad=True)


In [11]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [12]:
train_dataset = dataset(train_data,word_to_idx,label_to_ix)
test_dataset = dataset(test_data,word_to_idx,label_to_ix)
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
criterion = nn.NLLLoss()
def train_model(iter):
    for epoch in range(iter):
        for x,y in train_dataset:
            yhat = model(x)
            loss = criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()   

In [13]:
#in testing phase you must use torch.no_grad() or replace x with x.detach()
#because without doing it , gradient will track this part of code if training again it will take more time
#or maybe got wrong values
with torch.no_grad():
    for x,y in test_dataset:
        predict_before_train = model(x)
        print(predict_before_train)

print(next(model.parameters())[:, word_to_idx["creo"]])
train_model(100)

with torch.no_grad():
    for x,y in test_dataset:
        predict_after_train = model(x)
        print(predict_after_train)

print(next(model.parameters())[:, word_to_idx["creo"]])

tensor([[-0.6250, -0.7662]])
tensor([[-0.5870, -0.8119]])
tensor([0.0544, 0.1722], grad_fn=<SelectBackward>)
tensor([[-0.1210, -2.1721]])
tensor([[-2.7767, -0.0643]])
tensor([ 0.5004, -0.2738], grad_fn=<SelectBackward>)


In [14]:
#predict word "Give"
prediction_word = "que" #que / good / Give
tens =  next(model.parameters())[:, word_to_idx[prediction_word]]
value,index = tens.max(0)
print("word '",prediction_word,"' predicted as :", list(label_to_ix.keys())[index])  
#English 

word ' que ' predicted as : SPANISH
